In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font',family="Malgun Gothic")
plt.rcParams['axes.unicode_minus'] = False

df = pd.read_csv("Koweps_hpc10_2015_beta1.csv")
codebook = pd.read_excel("Koweps_Codebook.xlsx")

In [ ]:
# 컬럼명 전처리
new_column = dict(codebook[["변수명","설명"]].values)
df.rename(columns=new_column,inplace=True)

# 성별 전처리
gender_dataset = codebook['내용'][0].replace('.','').split()
df['성별'] = df['성별'].map({int(k):v for k,v in gender_dataset})

import re
marriage_data = codebook['내용'][2]
pattern = r'((\d+)\.(.*?)(?=\s\d+\.|$))' 
# (\d+)= \d: 숫자로시작, +:1개이상, (): 이 부분을 결과로 반환
# \.: 마침표찾기,
# (.*?) = .: 어떤 문자든 1개, *: 1개이상반복, ?: 가능한 짧게 
# (?=\s\d+\.|$) = (?= ...): lookahead라고 불리며, ...에 들어간 패턴이 뒤 따라오는 경우에만을 의미
# \s: 공백문자, \d+: 숫자하나이상, \.마침표, |: 또는 $: 문자열 끝
marriage_data = re.findall(pattern=pattern,string=marriage_data)
df['혼인상태'] = df["혼인상태"].map({int(k):v.strip() for s,k,v in marriage_data})
df[['성별','혼인상태']]

,성별,혼인상태
0,여,사별
1,여,사별
2,남,사별
3,남,이혼
4,여,사별
...,...,...
16659,여,유배우
16660,여,"미혼(18세이상, 미혼모 포함)"
16661,남,"미혼(18세이상, 미혼모 포함)"
16662,여,비해당(18세 미만)


### 1. 태어난 연도 데이터를 가지고 2015년기준으로 현재 나이를 나타내는 '나이' 열 df에 추가하기
### 2. 나이를 기준으로 (0 ~ 29살까지 청년), (30 ~ 59살까지를 중년), (60이상~ 노년)으로 표현하는 '연령대' 열 df에 추가하기
### 3. 각 연령대 별 남/여 성비 시각화(막대그래프, x=연령대,y=인원수), 남/여 막대그래프를 다른 색상으로 표현

In [9]:
df['나이'] = 2015- df['태어난 연도']
df['나이']

0        79
1        70
2        67
3        73
4        92
         ..
16659    48
16660    23
16661    20
16662    17
16663    14
Name: 나이, Length: 16664, dtype: int64

In [ ]:
age_range = [0,29,59,100]
age_labels = ['청'] 